# 2010~2023 재학생 충원율 데이터 전처리 템플릿

기준: 2023년에 존재하는 학교 + 사이버대학 제외

In [ ]:
import pandas as pd
from glob import glob
import os

In [ ]:
# 2023년 파일에서 유효한 학교명 기준 확보 (예시)
df_2023 = pd.read_excel("2023년 _대학_4-라-1. 재학생 충원율_학교별자료.xlsx", header=None)
valid_schools = df_2023[5].dropna().unique().tolist()
valid_schools = [s for s in valid_schools if "사이버" not in s]

In [ ]:
def process_yearly_file(file_path, year, school_col=5, quota_col=6, rate_col=12):
    df_raw = pd.read_excel(file_path, header=None)
    df_filtered = df_raw[[school_col, quota_col, rate_col]].copy()
    df_filtered.columns = ['학교', '편제 정원', '재학생 충원율(%)']

    # 정제
    df_filtered['편제 정원'] = pd.to_numeric(df_filtered['편제 정원'], errors='coerce')
    df_filtered['재학생 충원율(%)'] = pd.to_numeric(df_filtered['재학생 충원율(%)'], errors='coerce')
    df_filtered = df_filtered.dropna()
    df_filtered = df_filtered[df_filtered['학교'].isin(valid_schools)]
    df_filtered = df_filtered[~df_filtered['학교'].str.contains('사이버')]

    # 그룹별 가중 평균
    result = df_filtered.groupby('학교').apply(
        lambda g: pd.Series({
            '총 편제 정원': g['편제 정원'].sum(),
            '가중 평균 재학생 충원율(%)': (g['편제 정원'] * g['재학생 충원율(%)']).sum() / g['편제 정원'].sum()
        })
    ).reset_index()
    result['기준년도'] = year
    return result

In [ ]:
# 연도별 파일 처리
files = sorted(glob("????년 _대학_4-라-1. 재학생 충원율_학교별자료.xlsx"))
all_data = []

for file in files:
    year = int(os.path.basename(file)[:4])
    df = process_yearly_file(file, year)
    all_data.append(df)

# 통합
merged_df = pd.concat(all_data, ignore_index=True)
merged_df = merged_df[['기준년도', '학교', '총 편제 정원', '가중 평균 재학생 충원율(%)']]

# 저장
merged_df.to_csv("통합_재학생_충원율_2010_2023_CP949.csv", index=False, encoding="cp949")
merged_df.head()